In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Reference
https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722

In [3]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
# extract the whole website
soup = BeautifulSoup(website_url,'lxml')
# print(soup.prettify())

In [4]:
# extract the whole table
My_table = soup.find('table',{'class':'wikitable sortable'})

In [5]:
# extract all things about postcodes, borough and neighborhood
lists = My_table.findAll('td')
newlists = []
for item in lists:
    tmpstr = str(item)
    if tmpstr.find('</a>') == -1:
        newlists.append(tmpstr[tmpstr.find('<td>')+4:tmpstr.find('</td>')].replace('\n',''))
    else:
        newlists.append(tmpstr[tmpstr.find('">')+2:tmpstr.find('</a>')].replace('\n',''))
        
newlists[:6]

['M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned']

In [6]:
# extract columns
postcodes = newlists[0::3]
borough = newlists[1::3]
neighborhood = newlists[2::3]

In [7]:
neighborhood[-3:]

['Royal York South West', 'South of Bloor', 'Not assigned']

In [8]:
p,b,n = [],[],[]
for i in range(len(postcodes)):
    if borough[i] != 'Not assigned':
        p.append(postcodes[i])
        b.append(borough[i])
        if neighborhood[i] == 'Not assigned':
            n.append(borough[i])
        else:
            n.append(neighborhood[i])

In [9]:
from operator import itemgetter

In [14]:
POST,BOR,NEIGH = [],[],[]
for i in range(len(p)):
    if p[i] not in POST:
        POST.append(p[i])
        BOR.append(b[i])
        tmpid = [j for j,x in enumerate(p) if x==p[i]]
        tmgetter = itemgetter(*tmpid)
        tmplist = tmgetter(n)
        if isinstance(tmplist,str):
            NEIGH.append(tmplist)
        elif isinstance(tmplist,tuple):
            NEIGH.append(','.join(list(tmplist)))

In [15]:
df = pd.DataFrame()
df['PostalCode'] = POST
df['Borough'] = BOR
df['Neighborhood'] = NEIGH

#### Geographical coordinates of each postal code

#### Read csv

In [30]:
spadata = pd.read_csv('http://cocl.us/Geospatial_data')

#### Rename the column Postal Code

In [33]:
spadata.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

#### Merge two dataframes

In [34]:
df2 = pd.merge(df,spadata)

# Toronto neighborhoods

#### work with only boroughs that contain the word Toronto

In [40]:
df3 = df2[df2['Borough'].str.contains("Toronto")]
df3.shape

(38, 5)

In [66]:
print('The dataframe has {} boroughs.'.format(
        len(df3['Borough'].unique())))

The dataframe has 4 boroughs.


In [43]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [74]:
nei = df3.columns.difference(['Neighborhood'])
genres = df3['Neighborhood'].str.split(',')

df4 =  (df3.loc[df3.index.repeat(genres.str.len()),nei]
         .assign(Neighborhood=list(chain.from_iterable(genres.tolist())))).reset_index(drop=True)

In [75]:
df4.head()

,Borough,Latitude,Longitude,PostalCode,Neighborhood
0,Downtown Toronto,43.654260,-79.360636,M5A,Harbourfront
1,Downtown Toronto,43.654260,-79.360636,M5A,Regent Park
2,Downtown Toronto,43.657162,-79.378937,M5B,Ryerson
3,Downtown Toronto,43.657162,-79.378937,M5B,Garden District
4,Downtown Toronto,43.651494,-79.375418,M5C,St. James Town


In [65]:
df4.shape

(74, 5)

In [64]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df4['Borough'].unique()),
        df4.shape[0]))

The dataframe has 4 boroughs and 74 neighborhoods.


In [67]:
# create map of New York using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

#### Let's explore the first neighborhood in our dataframe.

In [76]:
df4.loc[0,'Neighborhood']

'Harbourfront'

##### So Let's explore Harbourfront

In [78]:
neighborhood_latitude = df4.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df4.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df4.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


In [80]:
radius = 500
LIMIT = 30
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [82]:
results = requests.get(url).json()

In [83]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [84]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [85]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

30 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

In [86]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [87]:
toronto_venues = getNearbyVenues(names=df4['Neighborhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [88]:
print(toronto_venues.shape)
toronto_venues.head()

(1644, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [89]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,30,30,30,30,30,30
Bathurst Quay,15,15,15,15,15,15
Berczy Park,30,30,30,30,30,30
Brockton,20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
CN Tower,15,15,15,15,15,15
Cabbagetown,30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
Chinatown,30,30,30,30,30,30


In [90]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 192 uniques categories.


## 3. Analyze Each Neighborhood

In [91]:
# one hot encoding
tor_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [92]:
tor_onehot.shape

(1644, 192)

In [93]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Adelaide,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.033333,0.000000,0.0,0.0,0.000000,0.0,0.0,0.066667,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.033333,0.0,0.0,0.0,0.000000,0.033333,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.00,0.00,0.033333,0.000000,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.000000,0.0,0.0,0.0,0.0,0.033333,0.00,0.033333,0.0,0.000000,0.00,0.00,0.0,0.066667,0.000000,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.033333,0.000000,0.0,0.033333,0.0,0.000000,0.033333,0.0,0.0,0.00,0.0,0.100000,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0
1,Bathurst Quay,0.000000,0.0,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.

#### Let's print each neighborhood along with the top 5 most common venues

In [94]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
              venue  freq
0        Steakhouse  0.10
1             Hotel  0.07
2       Pizza Place  0.07
3  Asian Restaurant  0.07
4              Café  0.07


----Bathurst Quay----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2    Airport Lounge  0.13
3   Harbor / Marina  0.07
4  Sculpture Garden  0.07


----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.07
1        Cocktail Bar  0.07
2                Café  0.07
3      Farmers Market  0.07
4            Beer Bar  0.07


----Brockton----
                    venue  freq
0             Coffee Shop  0.10
1                    Café  0.10
2          Breakfast Spot  0.10
3  Furniture / Home Store  0.05
4   Performing Arts Venue  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower----

In [95]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Steakhouse,Asian Restaurant,Café,Pizza Place,Hotel,Opera House,Coffee Shop,Noodle House,Concert Hall,Lounge
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique
2,Berczy Park,Farmers Market,Cocktail Bar,Café,Seafood Restaurant,Beer Bar,Jazz Club,Bakery,Italian Restaurant,Museum,Cheese Shop
3,Brockton,Coffee Shop,Breakfast Spot,Café,Furniture / Home Store,Caribbean Restaurant,Restaurant,Bar,Climbing Gym,Stadium,Italian Restaurant
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Pizza Place,Burrito Place,Brewery


## 4. Cluster Neighborhoods

In [98]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 1, 1, 2, 1, 1, 1, 1], dtype=int32)

In [103]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = df4

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

,Borough,Latitude,Longitude,PostalCode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,43.654260,-79.360636,M5A,Harbourfront,1,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Mexican Restaurant,Event Space,Dessert Shop,Farmers Market,Historic Site
1,Downtown Toronto,43.654260,-79.360636,M5A,Regent Park,1,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Mexican Restaurant,Event Space,Dessert Shop,Farmers Market,Historic Site
2,Downtown Toronto,43.657162,-79.378937,M5B,Ryerson,1,Café,Clothing Store,Japanese Restaurant,Beer Bar,Diner,Sandwich Place,Plaza,Hotel,Pizza Place,Spa
3,Downtown Toronto,43.657162,-79.378937,M5B,Garden District,1,Café,Clothing Store,Japanese Restaurant,Beer Bar,Diner,Sandwich Place,Plaza,Hotel,Pizza Place,Spa
4,Downtown Toronto,43.651494,-79.375418,M5C,St. James Town,1,Gastropub,Restaurant,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Hotel,Thai Restaurant


In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'],tor_merged['Longitude'],tor_merged['Neighborhood'],tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

#### cluster 1

In [106]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0,tor_merged.columns[[1] + list(range(5,tor_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,43.686412,0,Coffee Shop,Pub,Light Rail Station,Bagel Shop,American Restaurant,Sushi Restaurant,Athletics & Sports,Sports Bar,Pizza Place,Liquor Store
55,43.686412,0,Coffee Shop,Pub,Light Rail Station,Bagel Shop,American Restaurant,Sushi Restaurant,Athletics & Sports,Sports Bar,Pizza Place,Liquor Store
56,43.686412,0,Coffee Shop,Pub,Light Rail Station,Bagel Shop,American Restaurant,Sushi Restaurant,Athletics & Sports,Sports Bar,Pizza Place,Liquor Store
57,43.686412,0,Coffee Shop,Pub,Light Rail Station,Bagel Shop,American Restaurant,Sushi Restaurant,Athletics & Sports,Sports Bar,Pizza Place,Liquor Store
58,43.686412,0,Coffee Shop,Pub,Light Rail Station,Bagel Shop,American Restaurant,Sushi Restaurant,Athletics & Sports,Sports Bar,Pizza Place,Liquor Store


#### cluster 2

In [109]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1,tor_merged.columns[[1] + list(range(5,tor_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,43.654260,1,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Mexican Restaurant,Event Space,Dessert Shop,Farmers Market,Historic Site
1,43.654260,1,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Mexican Restaurant,Event Space,Dessert Shop,Farmers Market,Historic Site
2,43.657162,1,Café,Clothing Store,Japanese Restaurant,Beer Bar,Diner,Sandwich Place,Plaza,Hotel,Pizza Place,Spa
3,43.657162,1,Café,Clothing Store,Japanese Restaurant,Beer Bar,Diner,Sandwich Place,Plaza,Hotel,Pizza Place,Spa
4,43.651494,1,Gastropub,Restaurant,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Bakery,Gym,Hotel,Thai Restaurant
5,43.676357,1,Boutique,Health Food Store,Astrologer,Pub,Wings Joint,Cuban Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
6,43.644771,1,Farmers Market,Cocktail Bar,Café,Seafood Restaurant,Beer Bar,Jazz Club,Bakery,Italian Restaurant,Museum,Cheese Shop
7,43.657952,1,Coffee Shop,Italian Restaurant,Spa,Bubble Tea Shop,Café,Art Museum,Sandwich Place,Seafood Restaurant,Bar,Ramen Restaurant
8,43.669542,1,Grocery Store,Café,Park,Coffee Shop,Nightclub,Diner,Italian Restaurant,Convenience Store,Baby Store,Restaurant
9,43.650571,1,Steakhouse,Asian Restaurant,Café,Pizza Place,Hotel,Opera House,Coffee Shop,Noodle House,Concert Hall,Lounge


#### cluster 3

In [110]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2,tor_merged.columns[[1] + list(range(5,tor_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,43.628947,2,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique
60,43.628947,2,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique
61,43.628947,2,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique
62,43.628947,2,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique
63,43.628947,2,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique
64,43.628947,2,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique
65,43.628947,2,Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Boutique


#### cluster 4

In [111]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3,tor_merged.columns[[1] + list(range(5,tor_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,43.728020,3,Park,Dim Sum Restaurant,Bus Line,Swim School,Coworking Space,Dog Run,Discount Store,Diner,Dessert Shop,Deli / Bodega
34,43.696948,3,Park,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Coworking Space,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
35,43.696948,3,Park,Jewelry Store,Trail,Bus Line,Sushi Restaurant,Coworking Space,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
49,43.689574,3,Park,Gym,Playground,Coworking Space,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
50,43.689574,3,Park,Gym,Playground,Coworking Space,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
66,43.679563,3,Park,Trail,Playground,Cosmetics Shop,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega,Dance Studio


#### cluster 5

In [112]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4,tor_merged.columns[[1] + list(range(5,tor_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,43.711695,4,Garden,Music Venue,Wings Joint,Coworking Space,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
